In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
data = pd.read_csv("~/datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv")

data.columns= data.columns.str.lower()

In [2]:
#formatting boolean columns
data['partner'] = data['partner'].map({'Yes': True, 'No': False})
data['dependents'] = data['dependents'].map({'Yes': True, 'No': False})
data['phoneservice'] = data['phoneservice'].map({'Yes': True, 'No': False})
data['paperlessbilling'] = data['paperlessbilling'].map({'Yes': True, 'No': False})
data['churn'] = data['churn'].map({'Yes': True, 'No': False})
data['seniorcitizen'] = data['seniorcitizen'].map({0: False, 1: True})

In [3]:
#converting object columns to more workable data types
data['totalcharges'] = pd.to_numeric(data['totalcharges'], errors='coerce')

data = data.astype({'gender':'category','multiplelines':'category',
            'internetservice':'category','onlinesecurity':'category',
            'onlinebackup':'category','deviceprotection':'category',
            'techsupport':'category','streamingtv':'category',
            'streamingmovies':'category','contract':'category',
            'paymentmethod':'category'})

In [4]:
#cleaning up column names
data = data.rename(columns={'seniorcitizen':'senior_citizen', 'phoneservice':'phone_service',
 'multiplelines':'multiple_lines', 'internetservice':'internet_service',
 'onlinesecurity':'online_security', 'onlinebackup':'online_backup',
 'deviceprotection':'device_protection', 'techsupport':'tech_support',
 'streamingtv':'streaming_tv', 'streamingmovies':'streaming_movies',
 'paperlessbilling':'paperless_bill','paymentmethod':'payment_method',
 'monthlycharges':'monthly_charges', 'totalcharges':'total_charges'})


In [6]:
#create dictionary for categorical data
dic = {'gender':{}, 'senior_citizen':{}, 'partner':{}, 'dependents':{},
       'phone_service':{}, 'multiple_lines':{}, 'internet_service':{},
       'online_security':{}, 'online_backup':{}, 'device_protection':{}, 'tech_support':{},
       'streaming_tv':{}, 'streaming_movies':{}, 'contract':{}, 'paperless_bill':{},
       'payment_method':{}}

# compute churn rates and fill in dictionary for each measure
for c in dic:
    out = data[[c, 'churn']].groupby(c).mean()
    dic[c] = out.to_dict(orient='index')

# turn the dictionary into a df
churn_rates = pd.DataFrame.from_dict(dic, orient="index").stack().to_frame()
# format data 
churn_rates = pd.DataFrame(churn_rates[0].values.tolist(), index=churn_rates.index)

In [7]:
churn_rates = churn_rates.reset_index()
churn_rates = churn_rates.rename(columns={'level_0':'measure','level_1':'measure_category'})

In [8]:
# create dataframe subset for tenure
ten = data[['tenure','churn']].sort_values(by='tenure')
ten['cum_churn'] = ten['churn'].cumsum()

In [9]:
# create dataframe subset for monthly_charges
charges = data[['monthly_charges','churn']].sort_values(by='monthly_charges')

charges['cum_churn'] = charges['churn'].cumsum()
charges['avg_churn']= charges['churn'].rolling(100).mean()

In [12]:
data.to_csv(r'/home/alex/projects/telco_data.csv', index=False)
churn_rates.to_csv(r'/home/alex/projects/telco_churn.csv', index=False)
ten.to_csv(r'/home/alex/projects/telco_tenure.csv', index=False)
charges.to_csv(r'/home/alex/projects/telco_charges.csv', index=False)